In [1]:
import os
import json
from tqdm import tqdm
from fyp.crypto import Crypto
from fyp.db import User, UserInteractorRelationships, Database, db


In [2]:
crypto = Crypto()
database = Database(crypto)


In [3]:
def load_data(name):
    base = "/its/home/ep396/Documents/FYP/data/snowball_second/"
    e = base + f"encrypted_{name}.json"
    d = base + f"decrypted_{name}.json"

    crypto.age_decrypt_file(e, d)

    file = open(d, encoding='utf8')
    data = json.load(file)
    file.close()

    os.remove(d)

    return data


In [4]:
def simple_create_relations(user_id, follower_ids, counts):
    user_orm = User.select(User.id).where(User.twitter_user_id == user_id)
    for follower_id in follower_ids:
        follower_orm = User.select(User.id).where(User.twitter_user_id == follower_id)
        count = counts[(user_id, follower_id)]
        database.add_interactor_relation(user_orm, follower_orm, count)

In [5]:
def get_unique_repliers(users, repliers):
    unique_users = []
    relations = {}
    counts = {}

    for user in tqdm(users):
        captured_users = repliers[str(user)]
        relations[user] = []
        for captured_user in captured_users:
            if captured_user not in unique_users: unique_users.append(captured_user)
            if captured_user not in relations[user]: relations[user].append(captured_user)
            if (user, captured_user) not in counts:
                counts[(user, captured_user)] = 0
            counts[(user, captured_user)] += 1

    return unique_users, relations, counts


In [6]:
users = {
    int(crypto.fernet_decrypt(user.twitter_user_id)): user.id
    for user in User.select(User.id, User.twitter_user_id).where(User.hop == 1)
}

user_reverse = {value: key for key, value in users.items()}
user_twitter_ids = [user for user in users.keys()]


In [7]:
repliers = load_data("repliers")


In [8]:
reply_users, relations, counts = get_unique_repliers(user_twitter_ids, repliers)


100%|██████████| 4852/4852 [04:29<00:00, 17.97it/s]  


In [13]:
sum([len(relations) for user, relations in relations.items()])


177511

In [14]:
len(reply_users)


77047

In [9]:
if database.encrypted():
    database.encrypt_or_decrypt()


100%|██████████| 4882/4882 [01:59<00:00, 40.70it/s]


In [10]:
for user in tqdm(reply_users):
    database.add_user(user, False, 2)


100%|██████████| 77047/77047 [1:12:20<00:00, 17.75it/s]  


In [11]:
for user, followers in tqdm(relations.items()):
    simple_create_relations(user, followers, counts)


 42%|████▏     | 2036/4852 [1:02:20<30:58,  1.52it/s]  

In [ ]:
if not database.encrypted():
    database.encrypt_or_decrypt()


100%|██████████| 44413/44413 [18:10<00:00, 40.72it/s]   
